Figure out how to get the data from all the snotel sites into one dataframe

Thinking averate? Or median?

In [21]:
import pandas as pd
# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
from os import listdir
from os.path import isfile, join

# Get all the seasonal snotel reports
# could just use Stations Selected, but individual files might be easier

path = '../content/csv/snotel/east_slopes_north/' # TODO: your sub-area name here
def get_seasonal_snotel_files():
    files = [path + f for f in listdir(path) if isfile(join(path, f))]
    return files

# TODO: run this block and make sure you see all the files you just added below
get_seasonal_snotel_files()

['../content/csv/snotel/east_slopes_north/515_HartsPass.csv',
 '../content/csv/snotel/east_slopes_north/681_ParkCreekRidge.csv',
 '../content/csv/snotel/east_slopes_north/711_RainyPass.csv']

Load them all into dataframes

In [22]:
def get_seasonal_snotel_dataframes():
    dfs = [pd.read_csv(f, comment='#') for f in get_seasonal_snotel_files()]
    # Rename columns
    for df in dfs:
        air_temp_max_col = [col for col in df.columns if col.endswith('Air Temperature Maximum (degF)')][0]
        delta_snow_depth_col = [col for col in df.columns if col.endswith('Change In Snow Depth (in) Start of Day Values')][0]
        delta_swe_col = [col for col in df.columns if col.endswith('Change In Snow Water Equivalent (in) Start of Day Values')][0]
        delta_temp_min_col = [col for col in df.columns if col.endswith('Change In Air Temperature Minimum (degF)')][0]
        df.rename(columns={
            air_temp_max_col: 'Max Air Temp 24hr', 
            delta_snow_depth_col: 'Total Snowfall 24hr',
            delta_swe_col: 'Delta SWE 24hr',
            delta_temp_min_col: 'Min Temp Diff 48hr'}, inplace=True)
    return dfs


get_seasonal_snotel_dataframes()[0].head()

,Date,Max Air Temp 24hr,Total Snowfall 24hr,Delta SWE 24hr,Min Temp Diff 48hr
0,2022-11-01,29.7,7,0.6,-3.1
1,2022-11-02,24.3,-3,-0.2,-5.6
2,2022-11-03,21.7,1,-0.3,0.9
3,2022-11-04,35.2,0,0.6,4.2
4,2022-11-05,25.7,12,2.3,-4.0


Aggregate with average

In [23]:
def drop_non_numerical(snotel_df):
    """Drop columns that should not be part of the aggregation"""
    return snotel_df.drop(['Station Id', 'Station Name', 'Latitude', 'Longitude'], axis=1)

def set_up_dataframe_for_aggregation(dfs):
    # Make sure all the columns are the same
    assert all(len(dfs[0].columns.intersection(df.columns)) == dfs[0].shape[1] for df in dfs), \
        'All dataframes must have the same columns'   
    # Drop columns not wanted in aggregation
    # dfs = [drop_non_numerical(df) for df in dfs]
    # Set the index of each dataframe to be the date
    dfs = [df.set_index('Date') for df in dfs]
    # Concatenate all the dataframes
    cat = pd.concat(dfs)
    # Group them by their index
    by_index = cat.groupby(cat.index)

    return by_index

def get_average_seasonal_snotel_dataframe(dfs=get_seasonal_snotel_dataframes()):
    dfs = set_up_dataframe_for_aggregation(dfs)
    means = dfs.mean()
    return means

# dfs = get_seasonal_snotel_dataframes()
# dfs[0] = dfs[0].drop('Date', axis=1)
# should fail   
# get_seasonal_snotel_dataframes(dfs)

get_average_seasonal_snotel_dataframe().head()

,Max Air Temp 24hr,Total Snowfall 24hr,Delta SWE 24hr,Min Temp Diff 48hr
Date,,,,
2022-11-01,33.166667,2.333333,0.333333,-3.266667
2022-11-02,30.333333,-0.666667,0.066667,-4.200000
2022-11-03,27.166667,0.333333,-0.200000,-0.533333
2022-11-04,37.200000,2.000000,0.433333,4.133333
2022-11-05,30.266667,9.666667,1.600000,-4.033333


Aggregate with median

In [24]:
def get_median_seasonal_snotel_dataframe(dfs=get_seasonal_snotel_dataframes()):
    dfs = set_up_dataframe_for_aggregation(dfs)
    meds = dfs.median()
    return meds

get_median_seasonal_snotel_dataframe().head()

,Max Air Temp 24hr,Total Snowfall 24hr,Delta SWE 24hr,Min Temp Diff 48hr
Date,,,,
2022-11-01,31.3,0.0,0.3,-3.1
2022-11-02,29.5,0.0,-0.1,-5.0
2022-11-03,28.9,0.0,-0.2,-0.9
2022-11-04,36.3,1.0,0.5,4.2
2022-11-05,32.0,11.0,1.3,-4.0


Pop these dataframes into a file

Sorry Victor, probably could have just used report generator for this

In [6]:
means = get_average_seasonal_snotel_dataframe()
means.to_csv(path + 'seasonal_mean.csv', sep=',')

medians = get_median_seasonal_snotel_dataframe()
medians.to_csv(path + 'seasonal_median.csv', sep=',')